In [1]:
import datetime
from datetime import timedelta

In [2]:
class Degree:
    def __init__(self, degree, minute, second):
        self.degree = degree
        self.minute = minute
        self.second = second
        
    @property
    def degree_in_seconds(self):
        self._degree_in_seconds = self.degree * 3600 + self.minute * 60 + self.second
        return self._degree_in_seconds
    
    def __truediv__(self, x):
        return self.degree_in_seconds / x.degree_in_seconds
    
    def __sub__(self, x):
        diff = self.degree_in_seconds - x.degree_in_seconds
        d = diff // 3600
        m = (diff % 3600) // 60
        s = diff % 60
        return self.__class__(d, m, s)
    
    def __add__(self, x):
        sum = self.degree_in_seconds + x.degree_in_seconds
        d = sum // 3600
        m = (sum % 3600) // 60
        s = sum % 60
        return self.__class__(d, m, s)
    
    def __str__(self):
        return str(self.degree) + ',' + str(self.minute) + ',' +  str(self.second)
    
    def times(self, n):
        sum = self.degree_in_seconds * n
        d = sum // 3600
        m = (sum % 3600) // 60
        s = sum % 60
        return self.__class__(d, m, s)
    


In [3]:
def get_next_dasha_no(current_dasha_no):
    if (current_dasha_no + 1) % 9  != 0:
        return (current_dasha_no + 1) % 9 
    else:
        return 9

In [4]:
def get_all_dashas_in_seq(current_dasha_no):
    yield current_dasha_no
    while get_next_dasha_no(current_dasha_no) != current_dasha_no:
        yield get_next_dasha_no(current_dasha_no)
        current_dasha_no = get_next_dasha_no(current_dasha_no)

In [5]:
class Planet:
    def __init__(self, planet_name, rasi_number, rasi_degree, nakhetra_number, pada_number):
        self.planet_name = planet_name
        self.rasi_number = rasi_number
        self.rasi_degree = rasi_degree
        self.nakhetra_number = nakhetra_number
        self.pada_number = pada_number

In [6]:
mahadasha_days = dict()
mahadasha_days[6] = 18 / 120
mahadasha_days[7] = 16 / 120
mahadasha_days[8] = 19 / 120
mahadasha_days[9] = 17 / 120
mahadasha_days[1] = 7 / 120
mahadasha_days[2] = 20 / 120
mahadasha_days[3] = 6 / 120
mahadasha_days[4] = 10 / 120
mahadasha_days[5] = 7 / 120

In [7]:
class vimshottari:
    mahadasha_number_planet_mapping = {
            1: 'ketu',
            2: 'venus',
            3: 'sun',
            4: 'moon',
            5: 'mars',
            6: 'rahu',
            7: 'jupiter',
            8: 'saturn',
            9: 'mercury' }

    
    def __init__(self, date_of_birth, moon):
        self.date_of_birth = date_of_birth
        self.moon = moon
        self.total_vimshottari_dasha_seconds = 120 * 365 * 86400
        
    @property
    def mahadasha_no_at_birth(self):
        if self.moon.nakhetra_number % 9 == 0:
            return 9
        else:
            return self.moon.nakhetra_number % 9
        
    @property
    def dasha_degrees_remaining_fraction(self):
        tot_abs_degree =  Degree(13, 20, 0).times(self.moon.nakhetra_number)
        moon_abs_degree =  Degree(30, 0, 0).times(self.moon.rasi_number -1) + self.moon.rasi_degree
        #print(tot_abs_degree, moon_abs_degree)
        nakhetra_degree = Degree(13, 20, 0)
        return (tot_abs_degree - moon_abs_degree) / nakhetra_degree
    
    @property
    def dasha_degrees_over_fraction(self):
        return 1 - self.dasha_degrees_remaining_fraction
    
    @property
    def virtual_beginning_of_dasha(self):
        return self.date_of_birth - timedelta(seconds=self.dasha_degrees_over_fraction * self.total_vimshottari_dasha_seconds * mahadasha_days[self.mahadasha_no_at_birth])
        
    def find_dasha_running(self, date):
        self.dasha_running = []
        self.__find_dasha_running(date, self.mahadasha_no_at_birth, self.total_vimshottari_dasha_seconds, self.virtual_beginning_of_dasha)
        
    def __find_dasha_running(self, date, dasha_running, total_dasha, dasha_begins):
        dasha_period = mahadasha_days[dasha_running] * total_dasha
        dasha_ends = dasha_begins + timedelta(seconds=dasha_period)
        if len(self.dasha_running) < 5:
            if date > dasha_ends :
                dasha_begins = dasha_ends
                self.__find_dasha_running(date, get_next_dasha_no(dasha_running), total_dasha, dasha_begins)
            else:
                self.dasha_running.append(dasha_running)
                total_dasha = dasha_period
                self.__find_dasha_running(date, dasha_running, total_dasha, dasha_begins)
    
    def print_dasha_running(self):
        dasha_planets = []
        for number in self.dasha_running:
            dasha_planets.append(self.__class__.mahadasha_number_planet_mapping[number])
        print('-'.join(dasha_planets))
    
        

In [9]:
moon = Planet('moon', 7, Degree(10, 37, 17), 15, 2) # 7 is rasi number, 15 is nakhetra number, 2 is pada number
date_of_birth = datetime.datetime(1980, 8, 17, 8, 15, 0)

dasha = vimshottari(date_of_birth, moon)

event = datetime.datetime.today()
dasha.find_dasha_running(event)
dasha.print_dasha_running()


saturn-moon-rahu-venus-saturn


## Notes : In one cycle of moon , 120 years of vimshottari dasha is covered !!! 